# Analyses of Quantized Siren like NNs
---

In [1]:
from utils.libs import *

In [2]:
NOTIFICATIONS_ENABLED_VIA_TOAST = True
IS_DYNAMIC_QUANT = False

In [3]:
# Data To analize saved within a given dir path.
single_run_timestamp = '1605594448-849077'
date_run = '17-11-2020'
if IS_DYNAMIC_QUANT:
    single_run_path = f"D:\\data\\data_thesys\\dynamic_quant\\cameramen\\{single_run_timestamp}\\train"
else:
    single_run_path = f"D:\\data\\data_thesys\\cameramen\\{date_run}\\{single_run_timestamp}\\train"
    pass
# Check dir exists.
check_dir_exists(single_run_path)

True

In [4]:
# Filter desired files containing data to be investigated.
regex_filter = re.compile('result_comb_train(_)?(\d+)?\.txt$')
# regex_filter = re.compile('result_comb_train\.txt$')
# Fetch files into a list.
files_list = get_all_files_by_ext(dir_path = single_run_path, ext = 'txt', recursive_search = False, regex_filter = regex_filter)

files_list

[]

In [5]:
# Fetch data into a numpy array.
if len(files_list ) != 0:
    res_arr = laod_data_from_files_list(files_list)

In [6]:
if len(files_list ) != 0:
    res_arr[0]

In [7]:
# Put fetched data within pandas dataframe.
# columns = "#params;seed;hl;hf;mse;psnr;ssim;mse_dyn;psnr_dyn;ssim_dyn;eta".split(";")
# columns = "#params;seed;hl;hf;mse;psnr;ssim;eta".split(";")
columns = "arch_no_tmp;trial_no;hf;hl;seed;#params;#params_2;mse;psnr;ssim;mse_q;psnr_q;ssim_q;eta_t;eta_v;eta_q".split(";")
df_res = pd.DataFrame(res_arr, columns = columns).sort_values(by = ['hf', 'hl', 'seed'])
df_res.head(5)

NameError: name 'res_arr' is not defined

In [ ]:
df_res.tail(5)

In [ ]:
df_res.shape

In [ ]:
print(set(df_res['hf'].values))
print(set(df_res['hl'].values))
print(set(df_res['seed'].values))

In [8]:
option_pickle_filename = 'options.pickle'
pickel_full_path = os.path.join(f'{single_run_path}', f'{option_pickle_filename}')

if os.path.isfile(pickel_full_path):
    with open(pickel_full_path, "rb") as f:
        opt = pickle.load(f)
        pass
    pass
else:
    parser = argparse.ArgumentParser()
    
    # hf = list(map(int, "45".split(" ")))
    hf = list(set(df_res['hf'].values))
    parser.add_argument("--hidden_features", default=hf, dest = "hidden_features", required=False)
    
    # hl = list(map(int, "2 3 4 5 6 7 8 9".split(" ")))
    hl = list(set(df_res['hl'].values))
    parser.add_argument("--hidden_layers", default=hl, dest = "hidden_layers", required=False)
    
    lr = 0.0001
    parser.add_argument("--lr", default=lr, required = False, dest = "lr")
    
    # seeds = list(map(int, "0 42 123 1234 101".split(" ")))
    seeds = list(set(df_res['seed'].values))
    parser.add_argument("--seeds", default=seeds, dest = "seeds", required=False)
    
    num_attempts = 1
    parser.add_argument("--num_attempts", default=num_attempts, dest = "num_attempts", required=False)
    
    
    opt, unknown = parser.parse_known_args()
    pass
opt

Namespace(batch_size=[1], config_filepath=None, cuda=True, dynamic_quant=[], enable_tensorboard_logging=False, epochs_til_ckpt=50, evaluate=True, experiment_name='train', frequences=[], image_filepath=None, logging_root='../../../../results/cameramen', lr=[0.0001], n_hf=[64], n_hl=[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], num_attempts=1, num_epochs=[300000], quant_engine='fbgemm', seed=[0], sidelength=[256], verbose=0)

In [9]:
try:
    param_grid = dict(
        timemstamp=[single_run_timestamp],
        hf=opt.hidden_features,
        hl=opt.hidden_layers,
        seeds=opt.seeds,
        lr=[opt.lr]
    )
    data = list(ParameterGrid(param_grid))
except:
    param_grid = dict(
        timemstamp=[single_run_timestamp],
        hf=opt.n_hf,
        hl=opt.n_hl,
        seeds=opt.seed,
        lr=[opt.lr]
    )
    data = list(ParameterGrid(param_grid))
    pass

In [10]:
df = pd.DataFrame(data = data)
df.head(5)

,hf,hl,lr,seeds,timemstamp
0,64,5,[0.0001],0,1605594448-849077
1,64,6,[0.0001],0,1605594448-849077
2,64,7,[0.0001],0,1605594448-849077
3,64,8,[0.0001],0,1605594448-849077
4,64,9,[0.0001],0,1605594448-849077


In [11]:
n = len(set(df.seeds.values))
n, df.shape[0] / n

(1, 11.0)

In [12]:
archs_names = [f"arch_no_{no}" for no in range(0, df.shape[0])]
trials_names = [f"trial_no_{no}" for no in range(0, opt.num_attempts)]

param_grid = dict(
    archs_names=archs_names,
    trials_names=trials_names,
)
data = list(ParameterGrid(param_grid))
data = list(map(list, (map(operator.methodcaller('values'), data))))
data

[['arch_no_0', 'trial_no_0'],
 ['arch_no_1', 'trial_no_0'],
 ['arch_no_2', 'trial_no_0'],
 ['arch_no_3', 'trial_no_0'],
 ['arch_no_4', 'trial_no_0'],
 ['arch_no_5', 'trial_no_0'],
 ['arch_no_6', 'trial_no_0'],
 ['arch_no_7', 'trial_no_0'],
 ['arch_no_8', 'trial_no_0'],
 ['arch_no_9', 'trial_no_0'],
 ['arch_no_10', 'trial_no_0']]

In [13]:
df[['archs_names', 'trials_names']] = data

In [14]:
df

,hf,hl,lr,seeds,timemstamp,archs_names,trials_names
0,64,5,[0.0001],0,1605594448-849077,arch_no_0,trial_no_0
1,64,6,[0.0001],0,1605594448-849077,arch_no_1,trial_no_0
2,64,7,[0.0001],0,1605594448-849077,arch_no_2,trial_no_0
3,64,8,[0.0001],0,1605594448-849077,arch_no_3,trial_no_0
4,64,9,[0.0001],0,1605594448-849077,arch_no_4,trial_no_0
5,64,10,[0.0001],0,1605594448-849077,arch_no_5,trial_no_0
6,64,11,[0.0001],0,1605594448-849077,arch_no_6,trial_no_0
7,64,12,[0.0001],0,1605594448-849077,arch_no_7,trial_no_0
8,64,13,[0.0001],0,1605594448-849077,arch_no_8,trial_no_0
9,64,14,[0.0001],0,1605594448-849077,arch_no_9,trial_no_0


In [15]:
regex_filter = re.compile('.+\.pth$')
# Fetch files into a list.
files_list = get_all_files_by_ext(dir_path = single_run_path, ext = 'pth', recursive_search = True, regex_filter = regex_filter)

In [16]:
t = files_list[0]

In [17]:
os.path.split(os.path.split(t)[0])[1] == 'checkpoints'

True

In [18]:
os.path.split(os.path.split(files_list[0])[1])

('', 'model_final.pth')

In [19]:
file_list_sorted = []
for a_arch in df['archs_names'].values[:]:
    for a_file in files_list[:]:
        found = False
        a_file_tmp = copy.deepcopy(a_file)
        while True:
            a_file_tmp = os.path.split(a_file_tmp)
            # print(a_file_tmp)
            if len(a_file_tmp) == 0: break
            if len(a_file_tmp) == 2:
                if a_file_tmp[1] == '':
                    break
            dir_name = a_file_tmp[1]
            if dir_name == a_arch:
                found = True
                break
            a_file_tmp = a_file_tmp[0]
        if found == True:
            file_list_sorted.append(a_file)
            break
file_list_sorted

[Path('D:/data/data_thesys/cameramen/17-11-2020/1605594448-849077/train/arch_no_0/checkpoints/model_final.pth'),
 Path('D:/data/data_thesys/cameramen/17-11-2020/1605594448-849077/train/arch_no_1/checkpoints/model_final.pth'),
 Path('D:/data/data_thesys/cameramen/17-11-2020/1605594448-849077/train/arch_no_2/checkpoints/model_final.pth'),
 Path('D:/data/data_thesys/cameramen/17-11-2020/1605594448-849077/train/arch_no_3/checkpoints/model_final.pth'),
 Path('D:/data/data_thesys/cameramen/17-11-2020/1605594448-849077/train/arch_no_4/checkpoints/model_final.pth'),
 Path('D:/data/data_thesys/cameramen/17-11-2020/1605594448-849077/train/arch_no_5/checkpoints/model_final.pth'),
 Path('D:/data/data_thesys/cameramen/17-11-2020/1605594448-849077/train/arch_no_6/checkpoints/model_final.pth'),
 Path('D:/data/data_thesys/cameramen/17-11-2020/1605594448-849077/train/arch_no_7/checkpoints/model_final.pth'),
 Path('D:/data/data_thesys/cameramen/17-11-2020/1605594448-849077/train/arch_no_8/checkpoints/mo

In [20]:
df['path'] = file_list_sorted
df.head(5)

,hf,hl,lr,seeds,timemstamp,archs_names,trials_names,path
0,64,5,[0.0001],0,1605594448-849077,arch_no_0,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...
1,64,6,[0.0001],0,1605594448-849077,arch_no_1,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...
2,64,7,[0.0001],0,1605594448-849077,arch_no_2,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...
3,64,8,[0.0001],0,1605594448-849077,arch_no_3,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...
4,64,9,[0.0001],0,1605594448-849077,arch_no_4,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...


In [21]:
columns_list = list(df.columns)
RowTuple = collections.namedtuple('RowTuple', columns_list)

In [22]:
root_dir = f'D:\\data\\data_thesys\\data_post\\{date_run}\\{single_run_timestamp}'
def map_file_to_copy(a_row, root_dir = root_dir, ext = 'pth'):
    a_row_tuple = RowTuple._make(a_row)
    pieces = [a_row_tuple.timemstamp, a_row_tuple.hf, a_row_tuple.hl, a_row_tuple.archs_names,a_row_tuple.trials_names, ext]
    a_path = '.'.join([str(xx) for xx in pieces])
    return os.path.join(root_dir, a_path)
df['path_2'] = list(map(map_file_to_copy, df.values))

In [23]:
df.head(5)

,hf,hl,lr,seeds,timemstamp,archs_names,trials_names,path,path_2
0,64,5,[0.0001],0,1605594448-849077,arch_no_0,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...,D:\data\data_thesys\data_post\17-11-2020\16055...
1,64,6,[0.0001],0,1605594448-849077,arch_no_1,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...,D:\data\data_thesys\data_post\17-11-2020\16055...
2,64,7,[0.0001],0,1605594448-849077,arch_no_2,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...,D:\data\data_thesys\data_post\17-11-2020\16055...
3,64,8,[0.0001],0,1605594448-849077,arch_no_3,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...,D:\data\data_thesys\data_post\17-11-2020\16055...
4,64,9,[0.0001],0,1605594448-849077,arch_no_4,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...,D:\data\data_thesys\data_post\17-11-2020\16055...


In [24]:
df.iloc[0, -1]

'D:\\data\\data_thesys\\data_post\\17-11-2020\\1605594448-849077\\1605594448-849077.64.5.arch_no_0.trial_no_0.pth'

In [25]:
try:
    os.makedirs(root_dir)
except:
    print(f"Dir {root_dir} already exists!")
    pass

In [26]:
filename_csv = os.path.join(root_dir, 'create_collection.csv')
df.to_csv(filename_csv)

In [27]:
for src, dst in df[['path', 'path_2']].values:
    shutil.copyfile(src, dst)
    pass

In [28]:
df

,hf,hl,lr,seeds,timemstamp,archs_names,trials_names,path,path_2
0,64,5,[0.0001],0,1605594448-849077,arch_no_0,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...,D:\data\data_thesys\data_post\17-11-2020\16055...
1,64,6,[0.0001],0,1605594448-849077,arch_no_1,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...,D:\data\data_thesys\data_post\17-11-2020\16055...
2,64,7,[0.0001],0,1605594448-849077,arch_no_2,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...,D:\data\data_thesys\data_post\17-11-2020\16055...
3,64,8,[0.0001],0,1605594448-849077,arch_no_3,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...,D:\data\data_thesys\data_post\17-11-2020\16055...
4,64,9,[0.0001],0,1605594448-849077,arch_no_4,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...,D:\data\data_thesys\data_post\17-11-2020\16055...
5,64,10,[0.0001],0,1605594448-849077,arch_no_5,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...,D:\data\data_thesys\data_post\17-11-2020\16055...
6,64,11,[0.0001],0,1605594448-849077,arch_no_6,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...,D:\data\data_thesys\data_post\17-11-2020\16055...
7,64,12,[0.0001],0,1605594448-849077,arch_no_7,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...,D:\data\data_thesys\data_post\17-11-2020\16055...
8,64,13,[0.0001],0,1605594448-849077,arch_no_8,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...,D:\data\data_thesys\data_post\17-11-2020\16055...
9,64,14,[0.0001],0,1605594448-849077,arch_no_9,trial_no_0,D:\data\data_thesys\cameramen\17-11-2020\16055...,D:\data\data_thesys\data_post\17-11-2020\16055...


In [29]:
df_tmp = df[["hf", "hl", "lr", "seeds", "timemstamp", "archs_names", "trials_names"]]

In [30]:
def map_path(a_path, target = f"D:\\data\\data_thesys\\data_post\\{date_run}\\{single_run_timestamp}", update = f'\\content\\{single_run_timestamp}'):
    a_path_updated = a_path.replace(f"{target}", f"{update}").replace("\\", "/")
    return a_path_updated

df_tmp['path'] = list(map(map_path, df['path_2'].values))
df_tmp.head(5)

,hf,hl,lr,seeds,timemstamp,archs_names,trials_names,path
0,64,5,[0.0001],0,1605594448-849077,arch_no_0,trial_no_0,/content/1605594448-849077/1605594448-849077.6...
1,64,6,[0.0001],0,1605594448-849077,arch_no_1,trial_no_0,/content/1605594448-849077/1605594448-849077.6...
2,64,7,[0.0001],0,1605594448-849077,arch_no_2,trial_no_0,/content/1605594448-849077/1605594448-849077.6...
3,64,8,[0.0001],0,1605594448-849077,arch_no_3,trial_no_0,/content/1605594448-849077/1605594448-849077.6...
4,64,9,[0.0001],0,1605594448-849077,arch_no_4,trial_no_0,/content/1605594448-849077/1605594448-849077.6...


In [31]:
root_dir = f'D:\\data\\data_thesys\\data_post\\{date_run}\\{single_run_timestamp}'
filename_csv = os.path.join(root_dir, f'colab_{single_run_timestamp}.csv')
df_tmp.to_csv(filename_csv)

In [32]:
cropped_heigth = [256] * df_tmp.shape[0]
df_tmp['cropped_heigth'] = cropped_heigth
df_tmp['cropped_width'] = cropped_heigth

In [33]:
root_dir = f'D:\\data\\data_thesys\\data_post\\{date_run}\\{single_run_timestamp}'
filename_csv = os.path.join(root_dir, f'colab_{single_run_timestamp}.csv')
df_tmp.to_csv(filename_csv)

In [34]:
columns = "hf,hl,lr,seeds,timestamp,archs_names,trials_names,path,cropped_heigth,cropped_width".split(",")
df_tmp.columns = columns
filename_csv = os.path.join(root_dir, f'colab_{single_run_timestamp}.csv')
df_tmp.to_csv(filename_csv)

In [35]:
df_tmp.head(5)

,hf,hl,lr,seeds,timestamp,archs_names,trials_names,path,cropped_heigth,cropped_width
0,64,5,[0.0001],0,1605594448-849077,arch_no_0,trial_no_0,/content/1605594448-849077/1605594448-849077.6...,256,256
1,64,6,[0.0001],0,1605594448-849077,arch_no_1,trial_no_0,/content/1605594448-849077/1605594448-849077.6...,256,256
2,64,7,[0.0001],0,1605594448-849077,arch_no_2,trial_no_0,/content/1605594448-849077/1605594448-849077.6...,256,256
3,64,8,[0.0001],0,1605594448-849077,arch_no_3,trial_no_0,/content/1605594448-849077/1605594448-849077.6...,256,256
4,64,9,[0.0001],0,1605594448-849077,arch_no_4,trial_no_0,/content/1605594448-849077/1605594448-849077.6...,256,256


In [36]:
if NOTIFICATIONS_ENABLED_VIA_TOAST:
    from win10toast import ToastNotifier
    toaster = ToastNotifier()

    toaster.show_toast(
        "Notification", f"Data Prepared!\nDate: {date_run}\nTs: {single_run_timestamp}",
        threaded = True, icon_path=None, duration=3)
    pass